In [140]:
import re
import os
import json

# Removing the HTML tags from the code
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

pd.set_option('display.max_rows', None)

In [141]:
def generate_data(data):
    exercises = data['labs'][0]['exercises']

    id_with_exercises = []

    # Declaring certain types to not include
    for exercise in exercises:
        if(exercise['type'] == 'WLC' \
           or exercise['type'] == 'STI' \
           or exercise['type'] == 'DWS' \
           or exercise['type'] == 'RSLT' \
           or len(exercise['questions']) == 0):
            continue

        id = exercise['id']

        exercise_type = exercise['type']

        questions = exercise['questions']
        quiz_data = exercise['_data']
        quiz_flag = False

        if('quiz' in str(quiz_data).lower()):
            quiz_flag = True

        exercise_question_list = []
        exercise_correct_response_list = []
        exercise_incorrect_response_list = []
        exercise_type_list = []
        exercise_is_quiz = []

        for question in questions:
            contents = question['_data']
            responses = question['responses']
            Q = ''
            for content in contents:
                if('contents' in content):
                    if(content['type'] == 'question'):
                        Q += remove_tags(content['contents'][0]['content'])
                    elif(content['type'] == 'post_question'):
                        Q += remove_tags(content['contents']['content'])
            IR = ''
            for response in responses:
                if(response['correct'] == 1):
                    CR = response['_data'][0]['content']
                else:
                    IR += response['_data'][0]['content'] + ","

            exercise_question_list.append(Q)
            exercise_correct_response_list.append(CR)
            exercise_incorrect_response_list.append(IR)
            exercise_type_list.append(exercise_type)
            exercise_is_quiz.append(quiz_flag)
        
        # JSON structure with the required keys and values
        id_with_exercises.append({
            'exercise_id': id,
            'questions': exercise_question_list,
            'correct_responses': exercise_correct_response_list,
            'incorrect_responses': exercise_incorrect_response_list,
            'type': exercise_type_list,
            'is_quiz': exercise_is_quiz
        })

    return id_with_exercises

In [142]:
def make_dataframe(id_with_exercises):
    import pandas as pd
    df = pd.DataFrame.from_records(id_with_exercises)
    return df

In [143]:
# Directory for all the json data

directory = 'lesson_jsons'
data_frames = []

for root, dirs, files in os.walk(directory):
    for filename in files:
        file = os.path.join(root, filename)
        with open(file, encoding="utf-8") as json_data:
            data = json.load(json_data)
            id_with_exercises = generate_data(data)
            df = make_dataframe(id_with_exercises)
            data_frames.append(df)

In [144]:
result = pd.concat(data_frames)
result.reset_index(drop=True)
result.set_index(['exercise_id']).apply(pd.Series.explode).reset_index()

,exercise_id,questions,correct_responses,incorrect_responses,type,is_quiz
0,2,engenn2le02ob03re3aud01,"go for a walk, or play a sport.",",",MT1,False
1,2,engenn2le02ob03re3aud02,sunny outside.,",",MT1,False
2,2,engenn2le02ob03re3aud03,I stay home?,",",MT1,False
3,2,engenn2le02ob03re3aud04,beautiful day!,",",MT1,False
4,2,engenn2le02ob03re3aud05,to the beach.,",",MT1,False
5,3,Peter's phone number is ...,455 2134,"454 2143 ,455 3134 ,",DMC,False
6,3,John asks ...,Is this Peter Smith?,"Are you Smith? ,Peter, is that you?,",DMC,False
7,3,Today Peter is …,tired.,"out of town. ,at the beach.,",DMC,False
8,4,engenn2le02ob03re5aud01is shining.,sun,",",TYP,False
9,4,engenn2le02ob03re5aud02.,outside,",",TYP,False
